In [1]:
%load_ext autoreload
%autoreload 2

from jenga.tasks.openml import OpenMLTask
from jenga.cleaning.cleaners import AutoGluonCleaner, SklearnCleaner
from jenga.corruptions.numerical import Scaling
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.autocleaning import AutoCleaningEvaluator
import numpy as np

In [2]:
task = OpenMLTask(0)

cleaner = AutoGluonCleaner(numeric_columns=task.numerical_columns, 
                           categorical_columns=task.categorical_columns)

Found 0 categorical columns: []
Found 39 numeric columns: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'z', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'ar']


In [3]:
model = task.fit_baseline_model(task.train_data, task.train_labels)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.7s finished


In [4]:
# cleaner.fit(task.train_data)
# evaluator = AutoCleaningEvaluator(task, cleaner)

In [5]:
corruptions = []
for impacted_column in task.numerical_columns + task.categorical_columns:
    for fraction in [0.99, 0.5, 0.25, 0.1, 0.01]:
        if impacted_column in task.numerical_columns:
            corruptions.append(Scaling(impacted_column, fraction))
        for missingness in ['MCAR', 'MAR', 'MNAR']:
            corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
            corruptions.append(corruption)

In [6]:
# evaluator.evaluate(model, 5, *corruptions[:2])

In [7]:
sklearn_cleaner = SklearnCleaner(numeric_columns=task.numerical_columns, 
                           categorical_columns=task.categorical_columns)

In [8]:
sklearn_cleaner.fit(task.train_data)
sklearn_cleaner_evaluator = AutoCleaningEvaluator(task, sklearn_cleaner)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.2s finished


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s finished


Fitting 2 folds for each of 2 candidates, totalling 4 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.1s finished


In [ ]:
sklearn_cleaner_results = sklearn_cleaner_evaluator.evaluate(model, 5, *corruptions[:2])

In [19]:
sklearn_cleaner_results[1].__dict__

{'corruption': <jenga.corruptions.generic.MissingValues at 0x13aeedd10>,
 'cleaner': <jenga.cleaning.cleaners.SklearnCleaner at 0x13afa9550>,
 'baseline_score': 0.7999999999999999,
 'corrupted_scores': [0.8, 0.8, 0.8, 0.8, 0.8],
 'score_with_anomaly_removal': [0.5764705882352941,
  0.5764705882352941,
  0.5764705882352941,
  0.5764705882352941,
  0.5764705882352941],
 'score_with_imputation': [0.7941176470588236,
  0.7941176470588236,
  0.7941176470588236,
  0.7941176470588236,
  0.7941176470588236],
 'score_with_cleaning': [0.5705882352941176,
  0.5705882352941176,
  0.5705882352941176,
  0.5705882352941176,
  0.5705882352941176]}